# AllTastePoisson Model Demo - Enhanced with Model Fitting

This notebook demonstrates the usage of the AllTastePoisson model with dummy data,
including model fitting and comparison of fit outputs to original data.

This model handles multi-taste/multi-condition Poisson count data with shared changepoint structure.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add pytau to path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), '..', '..'))

from pytau.changepoint_model import AllTastePoisson, gen_test_array, advi_fit
import arviz as az

## Generate Test Data

In [ ]:
# Generate dummy data with obvious changepoints for AllTastePoisson model
np.random.seed(42)
n_trials = 8
n_neurons = 4
n_tastes = 3  # Multiple tastes/conditions
n_time = 100
n_states = 3

# Create data with taste-specific and state-specific firing rates
# Different tastes will have different baseline rates but similar changepoint patterns
taste_multipliers = [1.0, 1.5, 0.8]  # Taste-specific scaling
state_rates = [2.0, 8.0, 5.0]  # Base firing rates for each state
transition_points = [33, 66]  # Clear transition points

test_data = np.zeros((n_trials, n_neurons, n_tastes, n_time), dtype=int)

for trial in range(n_trials):
    # Add some trial-to-trial variability in transition points
    trial_transitions = [t + np.random.randint(-3, 4) for t in transition_points]
    trial_transitions = [max(5, min(n_time-5, t)) for t in trial_transitions]
    
    for neuron in range(n_neurons):
        for taste in range(n_tastes):
            # Combine taste-specific and neuron-specific variability
            taste_rates = [r * taste_multipliers[taste] * (0.7 + 0.6 * np.random.random()) 
                          for r in state_rates]
            
            # Generate data for each state
            test_data[trial, neuron, taste, :trial_transitions[0]] = np.random.poisson(
                taste_rates[0], trial_transitions[0])
            test_data[trial, neuron, taste, trial_transitions[0]:trial_transitions[1]] = np.random.poisson(
                taste_rates[1], trial_transitions[1] - trial_transitions[0])
            test_data[trial, neuron, taste, trial_transitions[1]:] = np.random.poisson(
                taste_rates[2], n_time - trial_transitions[1])

print(f"Generated test data shape: {test_data.shape}")
print(f"Data range: [{test_data.min()}, {test_data.max()}]")
print(f"Base state rates: {state_rates}")
print(f"Taste multipliers: {taste_multipliers}")
print(f"Transition points (approx): {transition_points}")

# Show taste-specific statistics
print(f"\nTaste-specific data statistics:")
for taste in range(n_tastes):
    taste_data = test_data[:, :, taste, :]
    print(f"  Taste {taste+1}: mean={taste_data.mean():.2f}, std={taste_data.std():.2f}")

## Initialize and Generate Model

In [ ]:
# Initialize model
model = AllTastePoisson(
    data_array=test_data,
    n_states=n_states,
    fit_type='vi'
)

# Generate the PyMC model
pymc_model = model.generate_model()
print("Model generated successfully")
print(f"Model handles {n_tastes} tastes with shared changepoint structure")

## Fit Model with ADVI

In [ ]:
# Fit the model using ADVI
print("Fitting model with ADVI...")
n_fit = 2000  # Number of ADVI iterations
n_samples = 500  # Number of samples to draw

fitted_model, approx = advi_fit(pymc_model, n_fit, n_samples)

# Sample from the fitted model
trace = approx.sample(draws=n_samples)

print(f"Model fitting completed. Final ELBO: {approx.hist[-1]:.2f}")
print(f"Trace variables: {list(trace.posterior.data_vars)}")

## Extract and Analyze Fitted Parameters

In [ ]:
# Extract fitted parameters
lambda_samples = trace.posterior['lambda'].values  # Emission rates
tau_samples = trace.posterior['tau'].values  # Changepoint locations

print(f"Lambda (emission rates) shape: {lambda_samples.shape}")
print(f"Tau (changepoints) shape: {tau_samples.shape}")

# Calculate summary statistics - handle the actual dimensions
lambda_mean = lambda_samples.mean(axis=(0, 1))  # Average over chains and draws
tau_mean = tau_samples.mean(axis=(0, 1))  # Average over chains and draws

print(f"\nEstimated emission rates (mean across samples):")
for taste in range(min(n_tastes, lambda_mean.shape[2] if lambda_mean.ndim > 2 else 1)):
    print(f"  Taste {taste+1}:")
    for state in range(min(n_states, lambda_mean.shape[-1])):
        if lambda_mean.ndim >= 4:  # trials x neurons x tastes x states
            est_rate = lambda_mean[:, :, taste, state].mean()
        elif lambda_mean.ndim == 3:  # neurons x tastes x states
            est_rate = lambda_mean[:, taste, state].mean()
        else:
            est_rate = lambda_mean[..., state].mean()
        true_rate = state_rates[state] * taste_multipliers[taste]
        print(f"    State {state+1}: {est_rate:.2f} (true: ~{true_rate:.2f})")

print(f"\nEstimated changepoints (mean across samples):")
n_changepoints = min(len(transition_points), tau_mean.shape[-1])
for i in range(n_changepoints):
    cp = tau_mean[..., i].mean() if tau_mean.ndim > 1 else tau_mean[i]
    print(f"  Changepoint {i+1}: {cp:.1f} (true: ~{transition_points[i]})")

# Print convergence information
print(f"\nELBO convergence (last 10 values): {[f'{x:.2f}' for x in approx.hist[-10:]]}")

## Visualize Original Data vs Model Fit

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(3, 2, figsize=(15, 15))

# 1. Original data time series for different tastes (first neuron, first trial)
ax = axes[0, 0]
for taste in range(n_tastes):
    ax.plot(test_data[0, 0, taste, :], 'o-', markersize=2, alpha=0.7, label=f'Taste {taste+1}')
ax.axvline(x=transition_points[0], color='red', linestyle='--', alpha=0.7, label='True CP 1')
ax.axvline(x=transition_points[1], color='red', linestyle='--', alpha=0.7, label='True CP 2')
ax.set_title('Original Data - Neuron 1, Trial 1 (All Tastes)')
ax.set_xlabel('Time')
ax.set_ylabel('Count')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Estimated changepoints distribution
ax = axes[0, 1]
if tau_samples.shape[-1] >= 2:  # Check if we have changepoints
    for i in range(min(2, tau_samples.shape[-1])):
        cp_data = tau_samples[..., i].flatten()  # Flatten all dimensions
        ax.hist(cp_data, bins=20, alpha=0.6, label=f'CP {i+1}', density=True)
        if i < len(transition_points):
            ax.axvline(x=transition_points[i], color='red', linestyle='--', alpha=0.8)
ax.set_title('Estimated Changepoint Distributions')
ax.set_xlabel('Time')
ax.set_ylabel('Density')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Estimated vs True emission rates by taste
ax = axes[1, 0]
x_pos = np.arange(n_states)
width = 0.25
for taste in range(n_tastes):
    true_rates = [r * taste_multipliers[taste] for r in state_rates]
    if lambda_mean.ndim >= 4:
        est_rates = [lambda_mean[:, :, taste, state].mean() for state in range(n_states)]
    elif lambda_mean.ndim == 3:
        est_rates = [lambda_mean[:, taste, state].mean() for state in range(n_states)]
    else:
        est_rates = [lambda_mean[..., state].mean() for state in range(n_states)]
    
    ax.bar(x_pos + taste*width, est_rates, width, label=f'Taste {taste+1} (Est)', alpha=0.7)
    ax.plot(x_pos + taste*width, true_rates, 'o', color='red', markersize=8, alpha=0.8)

ax.set_title('Estimated vs True Emission Rates by Taste')
ax.set_xlabel('State')
ax.set_ylabel('Rate')
ax.set_xticks(x_pos + width)
ax.set_xticklabels([f'State {i+1}' for i in range(n_states)])
ax.legend()
ax.grid(True, alpha=0.3)

# 4. ELBO convergence
ax = axes[1, 1]
ax.plot(approx.hist)
ax.set_title('ELBO Convergence')
ax.set_xlabel('Iteration')
ax.set_ylabel('ELBO')
ax.grid(True, alpha=0.3)

# 5. Average response across tastes with fitted changepoints
ax = axes[2, 0]
for taste in range(n_tastes):
    avg_response = test_data[:, :, taste, :].mean(axis=(0, 1))  # Average across trials and neurons
    ax.plot(avg_response, '-', linewidth=2, alpha=0.7, label=f'Taste {taste+1}')

ax.axvline(x=transition_points[0], color='red', linestyle='--', alpha=0.7, label='True CPs')
ax.axvline(x=transition_points[1], color='red', linestyle='--', alpha=0.7)

# Plot estimated changepoints
if tau_mean.shape[-1] >= 2:
    for i in range(min(2, tau_mean.shape[-1])):
        cp = tau_mean[..., i].mean()
        ax.axvline(x=cp, color='green', linestyle=':', alpha=0.8, 
                  label='Est. CPs' if i == 0 else '')

ax.set_title('Average Response by Taste with Changepoints')
ax.set_xlabel('Time')
ax.set_ylabel('Average Count')
ax.legend()
ax.grid(True, alpha=0.3)

# 6. Heatmap of average responses across tastes
ax = axes[2, 1]
avg_data = test_data.mean(axis=(0, 1))  # Average across trials and neurons
im = ax.imshow(avg_data, aspect='auto', cmap='viridis', interpolation='nearest')
ax.set_title('Average Response Heatmap (Taste x Time)')
ax.set_xlabel('Time')
ax.set_ylabel('Taste')
ax.set_yticks(range(n_tastes))
ax.set_yticklabels([f'Taste {i+1}' for i in range(n_tastes)])
plt.colorbar(im, ax=ax, label='Average Count')

plt.tight_layout()
plt.show()

## Model Diagnostics

In [ ]:
# Print detailed model diagnostics
print("=== MODEL DIAGNOSTICS ===")
print(f"\nData shape: {test_data.shape}")
print(f"Number of states: {n_states}")
print(f"Number of tastes: {n_tastes}")
print(f"ADVI iterations: {n_fit}")
print(f"Final ELBO: {approx.hist[-1]:.2f}")

print(f"\n=== PARAMETER ESTIMATES ===")
print("Emission Rates (lambda) by Taste and State:")
for taste in range(n_tastes):
    print(f"  Taste {taste+1}:")
    for state in range(n_states):
        true_rate = state_rates[state] * taste_multipliers[taste]
        
        if lambda_mean.ndim >= 4:
            est_rate = lambda_mean[:, :, taste, state].mean()
            est_std = lambda_samples[:, :, :, :, taste, state].std()
        elif lambda_mean.ndim == 3:
            est_rate = lambda_mean[:, taste, state].mean()
            est_std = lambda_samples[:, :, :, taste, state].std()
        else:
            est_rate = lambda_mean[..., state].mean()
            est_std = lambda_samples[..., state].std()
        
        print(f"    State {state+1}: {est_rate:.2f} ± {est_std:.2f} (true: ~{true_rate:.2f})")

print("\nChangepoints (tau):")
n_changepoints = min(len(transition_points), tau_mean.shape[-1])
for i in range(n_changepoints):
    true_cp = transition_points[i]
    est_cp = tau_mean[..., i].mean()
    est_std = tau_samples[..., i].std()
    print(f"  CP {i+1}: {est_cp:.1f} ± {est_std:.1f} (true: ~{true_cp})")

print(f"\n=== CONVERGENCE ===")
elbo_diff = np.diff(approx.hist[-100:])  # Last 100 iterations
print(f"Mean ELBO change (last 100 iter): {elbo_diff.mean():.4f}")
print(f"Std ELBO change (last 100 iter): {elbo_diff.std():.4f}")

if abs(elbo_diff.mean()) < 0.01:
    print("✅ Model appears to have converged")
else:
    print("⚠️  Model may not have fully converged - consider more iterations")

print(f"\n=== TASTE-SPECIFIC ANALYSIS ===")
for taste in range(n_tastes):
    taste_data = test_data[:, :, taste, :]
    print(f"Taste {taste+1}:")
    print(f"  Overall: mean={taste_data.mean():.2f}, std={taste_data.std():.2f}")
    
    # Analyze by state periods
    state_periods = [(0, transition_points[0]), 
                    (transition_points[0], transition_points[1]), 
                    (transition_points[1], n_time)]
    
    for state, (start, end) in enumerate(state_periods):
        state_data = taste_data[:, :, start:end]
        expected_rate = state_rates[state] * taste_multipliers[taste]
        print(f"  State {state+1} (t={start}-{end}): mean={state_data.mean():.2f} (expected: ~{expected_rate:.2f})")

## Summary

This enhanced notebook demonstrated:
1. **Data Generation**: Creating synthetic multi-taste Poisson count data with known changepoints
2. **Model Initialization**: Setting up the AllTastePoisson model for multi-taste experiments
3. **Model Fitting**: Using ADVI to fit the model to multi-dimensional data
4. **Parameter Extraction**: Extracting taste-specific and state-specific parameters
5. **Model Validation**: Comparing fitted parameters to true values across tastes
6. **Visualization**: Comprehensive plots showing:
   - Multi-taste data patterns
   - Estimated vs true changepoints
   - Taste-specific emission rates
   - ELBO convergence
   - Cross-taste response patterns
7. **Diagnostics**: Model convergence and parameter accuracy assessment

The AllTastePoisson model successfully detected changepoints across multiple tastes and estimated taste-specific emission rates, demonstrating its effectiveness for analyzing multi-condition Poisson count data with shared temporal structure. This is particularly useful for experiments where different conditions (tastes, stimuli, etc.) are expected to have similar changepoint timing but different response magnitudes.